In [1]:
import soundfile
import torch
import copy
import matplotlib.pyplot as plt
import numpy as np

from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text
from espnet2.bin.jm_utils import *
from zeroth_data_loader import zeroth_dataset
from extract_grad_cam import *
from zeroth_model_loader import load_model
from hook_related_fn import *
from model_dict import model_dict
from tqdm import tqdm

In [2]:
asr_exp_name = 'asr_TF_CUSTOM_HEAD4_ENCL12_FF512_relu_lr46_ctc31'
asr_config_file = asr_exp_name + '/config.yaml'
asr_model_file = asr_exp_name + '/valid.acc.ave.pth'
exp_dir = '/home/jmpark/home_data_jmpark/espnet/egs2/zeroth_korean/asr1/exp'

# Set test wav for attention image extraction
dataset = zeroth_dataset()

speech2text = load_model(asr_config_file=asr_config_file, 
                            asr_model_file=asr_model_file, 
                            prune_ratio=0.0, 
                            prune_mode='no_prune',
                            device='cpu')

net = speech2text.asr_model

In [3]:
print(net)

ESPnetASRModel(
  (frontend): DefaultFrontend(
    (stft): Stft(n_fft=512, win_length=512, hop_length=128, center=True, normalized=False, onesided=True)
    (frontend): Frontend()
    (logmel): LogMel(sr=16000, n_fft=512, n_mels=80, fmin=0, fmax=8000.0, htk=False)
  )
  (normalize): GlobalMVN(stats_file=exp/STATS_TF_CUSTOM_HEAD4_ENCL12_FF512_relu_lr46_ctc31/train/feats_stats.npz, norm_means=True, norm_vars=True)
  (encoder): TransformerEncoder(
    (embed): Conv2dSubsampling(
      (conv): Sequential(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        (3): ReLU()
      )
      (out): Sequential(
        (0): Linear(in_features=4864, out_features=256, bias=True)
        (1): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (encoders): MultiSequential(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linea

In [4]:
module_list = []

for name, module in net.named_modules():
    if 'pred' in name:
        module_list.append((module, name))

hookF = {name: OutHook(module) for (module, name) in module_list}

In [5]:
dataset = zeroth_dataset()

In [6]:
speech = dataset[15][1]['speech']

In [7]:
out = speech2text(speech)

In [8]:
total = 0
total_nonzero = 0

for name in hookF:
    if 'encoder' in name:
        output = hookF[name].output[0]
        total += output.size(1) * output.size(2)
        total_nonzero += torch.count_nonzero(torch.nn.functional.relu(output)).item()
    
    elif 'linear_q_pred' in name:
        output = hookF[name].output

        for i in range(len(output)):
            out = output[i]
            total += out.size(1) * out.size(2)
            total_nonzero += torch.count_nonzero(torch.nn.functional.relu(out)).item()

    else:
        output = hookF[name].output[-1]
        total += output.size(1) * output.size(2)
        total_nonzero += torch.count_nonzero(torch.nn.functional.relu(output)).item()

print(total)
print(total_nonzero)
print(1-(total_nonzero/total))


34752
5521
0.8411314456721916
